# Chapter 3 Exercise 1: Meta-Learners - 因果效应估计的瑞士军刀

## 学习目标

1. 理解 S-Learner 和 T-Learner 的原理
2. 实现基础的 Meta-Learner 算法
3. 理解不同方法的优缺点
4. 掌握 CATE (条件平均处理效应) 估计和评估

---

## 什么是 Meta-Learners?

想象你是一位医生，想知道某种新药对不同病人的效果。问题是：每个病人只能选择吃药或不吃药中的一种，你永远无法同时观察到两种结果。

**Meta-Learners** 就是用 "元学习" 的思想来解决这个问题——它们把因果效应估计转化为监督学习问题，让我们可以用任何机器学习模型（如 Random Forest、XGBoost）来估计处理效应！

这就像是给你一套万能工具，可以把各种 ML 模型变成因果推断工具。

## 生活化类比：餐厅会员卡的效果

假设你经营一家餐厅，推出了会员卡（处理 T）：
- **处理组 (T=1)**：拿到会员卡的顾客
- **控制组 (T=0)**：没有会员卡的顾客
- **结果 Y**：月消费金额

你想知道：**会员卡到底让顾客多花了多少钱？** 而且更进一步：**对哪些顾客效果最好？**

| 顾客类型 | 原本消费 | 有会员卡后消费 | 处理效应 |
|---------|---------|--------------|--------|
| 学生党 | 200 | 350 | **+150** (效果好!) |
| 上班族 | 500 | 520 | +20 (效果一般) |
| 土豪 | 2000 | 2010 | +10 (几乎没效果) |

这种 **因人而异** 的效应就叫做 **CATE (Conditional Average Treatment Effect)**——条件平均处理效应。

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from typing import Tuple

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

np.random.seed(42)
print("环境准备完成！")

---

## Part 1: S-Learner (Single Model Learner)

### 核心思想

S-Learner 是最简单的 Meta-Learner：

**把处理 T 当作普通特征，训练一个模型同时预测所有人的结果**

$$\hat{\mu}(x, t) = \hat{E}[Y | X=x, T=t]$$

然后，CATE 就是预测差值：

$$\hat{\tau}(x) = \hat{\mu}(x, 1) - \hat{\mu}(x, 0)$$

### 类比：一个班级，一位老师

想象一个班级里有男生和女生，老师要预测他们的考试成绩：

- **S-Learner 方法**：老师把 "性别" 作为一个特征，用同一套模型预测所有学生的成绩
- 然后比较：如果这个学生是男生会考多少分？如果是女生会考多少分？差值就是 "性别效应"

### S-Learner 的流程图

```
训练阶段:
┌─────────────────┐      ┌─────────────────┐
│  特征 X + 处理 T │ ───► │  单一模型 f(X,T) │ ───► 预测 Y
└─────────────────┘      └─────────────────┘

预测阶段:
                         ┌─ f(X, T=1) ─┐
┌──────┐                 │             │    减法
│ 特征 X│ ───────────────┤             ├─────────► CATE
└──────┘                 │             │
                         └─ f(X, T=0) ─┘
```

In [ ]:
# ============================================================
# 🎯 练习 1.1: 实现 S-Learner
# ============================================================
# 目标: 实现 S-Learner，将处理 T 作为特征，训练单一模型
# 提示:
#   1. fit() 方法: 将 T 和 X 合并为特征矩阵，训练模型预测 Y
#   2. predict_cate() 方法: 分别预测 T=1 和 T=0 的结果，计算差值
# ============================================================

class SimpleSLearner:
    """
    S-Learner (Single Model Learner)
    
    核心思想:
    - 将处理 T 作为一个特征
    - 训练单一模型: Y = f(X, T)
    - CATE 估计: tau(x) = f(x, 1) - f(x, 0)
    """
    
    def __init__(self):
        # 初始化基础模型 - 使用随机森林
        self.model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 S-Learner
        
        TODO: 
        1. 将 T 和 X 合并为特征矩阵 (T 作为最后一列)
        2. 训练模型预测 Y
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE (个体处理效应)
        
        TODO:
        1. 构造 T=1 的特征: 把 1 添加到 X 后面
        2. 构造 T=0 的特征: 把 0 添加到 X 后面
        3. 分别预测 Y(1) 和 Y(0)
        4. 返回差值: Y(1) - Y(0)
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===

In [ ]:
# ============================================================
# 🎯 练习 1.2: 实现 T-Learner
# ============================================================
# 目标: 实现 T-Learner，分别为处理组和控制组训练两个模型
# 提示:
#   1. fit() 方法: 按 T 分组，分别训练两个模型
#   2. predict_cate() 方法: 使用两个模型分别预测，计算差值
# ============================================================

class SimpleTLearner:
    """
    T-Learner (Two Model Learner)
    
    核心思想:
    - 分别为处理组和控制组训练两个模型
    - mu_0(x) = E[Y|X=x, T=0]  (控制组模型)
    - mu_1(x) = E[Y|X=x, T=1]  (处理组模型)
    - CATE 估计: tau(x) = mu_1(x) - mu_0(x)
    """
    
    def __init__(self):
        # 初始化两个模型
        self.model_0 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
        self.model_1 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=43)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 T-Learner
        
        TODO:
        1. 将数据按 T 分成两组
        2. 用控制组数据训练 model_0
        3. 用处理组数据训练 model_1
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE
        
        TODO:
        1. 使用 model_1 预测 Y(1)
        2. 使用 model_0 预测 Y(0)
        3. 返回差值 Y(1) - Y(0)
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===

In [ ]:
# ============================================================
# 🎯 练习 1.3: 生成 Uplift 数据
# ============================================================
# 目标: 生成包含异质性处理效应的模拟数据
# 提示:
#   1. 生成特征 X1, X2 (标准正态分布)
#   2. 随机分配 T (伯努利分布, p=0.5)
#   3. 计算真实 CATE: 如果 heterogeneous=True, tau = 2 + X1 - 0.5*X2
#   4. 生成 Y0 = 5 + 2*X1 + X2 + noise
#   5. 生成 Y1 = Y0 + tau
#   6. 观测 Y = T * Y1 + (1-T) * Y0
# ============================================================

def generate_simple_uplift_data(
    n: int = 1000,
    heterogeneous: bool = True,
    seed: int = 42
) -> Tuple[pd.DataFrame, np.ndarray]:
    """
    生成简单的 Uplift 数据
    
    数据生成过程 (DGP):
    - X1, X2 ~ N(0, 1)
    - T ~ Bernoulli(0.5)  (随机分配)
    - Y(0) = 5 + 2*X1 + X2 + noise
    - Y(1) = Y(0) + tau(X)
    
    其中:
    - 如果 heterogeneous=True: tau(X) = 2 + X1 - 0.5*X2  (异质性效应)
    - 如果 heterogeneous=False: tau(X) = 2  (常数效应)
    
    Returns:
        (DataFrame, true_cate)
        DataFrame columns: X1, X2, T, Y
        true_cate: 真实的个体处理效应
    """
    # === 你的代码开始 ===
    
    pass  # 删除此行，填入你的代码
    
    # === 你的代码结束 ===

# 测试数据生成
df, true_cate = generate_simple_uplift_data(n=100)
if df is not None and true_cate is not None:
    print(f"样本量: {len(df)}")
    print(f"真实平均 CATE (ATE): {true_cate.mean():.4f}")
    print(f"CATE 范围: [{true_cate.min():.2f}, {true_cate.max():.2f}]")
    print("\n数据预览:")
    print(df.head().to_string())
else:
    print("[未完成] 请完成 generate_simple_uplift_data 函数")

---

## Part 2: 可视化与评估

让我们实现 CATE 评估指标和可视化效果！

In [ ]:
# ============================================================
# 🎯 练习 1.4: 评估 CATE 估计
# ============================================================
# 目标: 实现 CATE 估计质量的评估指标
# 提示:
#   1. MSE: mean((true - pred)^2)
#   2. MAE: mean(|true - pred|)
#   3. Correlation: np.corrcoef(true, pred)[0, 1]
#   4. PEHE: sqrt(MSE)
# ============================================================

def evaluate_cate_estimation(
    true_cate: np.ndarray,
    predicted_cate: np.ndarray
) -> dict:
    """
    评估 CATE 估计的质量
    
    计算以下指标:
    1. MSE: 均方误差 = mean((true - pred)^2)
    2. MAE: 平均绝对误差 = mean(|true - pred|)
    3. Correlation: 与真实 CATE 的相关系数
    4. PEHE: sqrt(MSE)
    
    Returns:
        字典包含所有指标
    """
    # === 你的代码开始 ===
    
    pass  # 删除此行，填入你的代码
    
    # === 你的代码结束 ===

# 测试评估函数
if 'true_cate' in locals() and true_cate is not None:
    # 用一个随机预测测试
    random_pred = np.random.randn(len(true_cate)) * 2 + 2
    metrics = evaluate_cate_estimation(true_cate, random_pred)
    if metrics and 'MSE' in metrics:
        print("随机预测的评估结果:")
        for key, value in metrics.items():
            print(f"  {key}: {value:.4f}")
    else:
        print("[未完成] 请完成 evaluate_cate_estimation 函数")
else:
    print("[提示] 请先完成练习 1.3 生成数据")

---

## 总结

### 今天学到了什么？

| 概念 | 要点 |
|------|------|
| **Meta-Learners** | 把因果推断问题转化为监督学习问题的通用方法 |
| **S-Learner** | 单模型，把 T 当特征，简单但可能低估效应 |
| **T-Learner** | 双模型，分别建模，灵活但可能方差大 |
| **CATE** | 条件平均处理效应，反映效应的异质性 |
| **PEHE** | 评估 CATE 估计的黄金标准 |

### 下一步

在下一个练习中，我们将学习 **Uplift Tree** —— 一种专门为因果推断设计的决策树！

---

*恭喜你完成了 Meta-Learners 的学习！* 🎉

In [ ]:
# ============================================================
# 🎯 练习 1.5: 对比 S-Learner 和 T-Learner
# ============================================================
# 目标: 实现函数来对比两种方法的性能
# 提示:
#   1. 生成数据
#   2. 训练 S-Learner 并预测
#   3. 训练 T-Learner 并预测
#   4. 评估两者，返回对比结果
# ============================================================

def compare_s_and_t_learner(
    n_samples: int = 2000,
    heterogeneous: bool = True
) -> Tuple[pd.DataFrame, np.ndarray, np.ndarray, np.ndarray]:
    """
    比较 S-Learner 和 T-Learner 的性能
    
    Returns:
        (comparison_df, s_pred, t_pred, true_cate)
    """
    # === 你的代码开始 ===
    
    pass  # 删除此行，填入你的代码
    
    # === 你的代码结束 ===

# 运行对比实验
print("=" * 50)
print("场景 1: 异质性效应")
print("=" * 50)
result = compare_s_and_t_learner(n_samples=1000, heterogeneous=True)
if result and result[0] is not None:
    comparison_df, s_pred, t_pred, true_cate = result
    print(comparison_df.to_string())
else:
    print("[未完成] 请完成所有练习后重新运行")

print("\n" + "=" * 50)
print("场景 2: 常数效应")
print("=" * 50)
result2 = compare_s_and_t_learner(n_samples=1000, heterogeneous=False)
if result2 and result2[0] is not None:
    print(result2[0].to_string())

In [ ]:
# 可视化 CATE 估计结果

if 'result' in locals() and result and result[0] is not None:
    _, s_pred, t_pred, true_cate = result
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 图1: 真实 vs S-Learner
    axes[0].scatter(true_cate, s_pred, alpha=0.5, s=10)
    axes[0].plot([true_cate.min(), true_cate.max()], 
                 [true_cate.min(), true_cate.max()], 'r--', lw=2)
    axes[0].set_xlabel('True CATE')
    axes[0].set_ylabel('Predicted CATE')
    axes[0].set_title('S-Learner: True vs Predicted')
    
    # 图2: 真实 vs T-Learner
    axes[1].scatter(true_cate, t_pred, alpha=0.5, s=10)
    axes[1].plot([true_cate.min(), true_cate.max()], 
                 [true_cate.min(), true_cate.max()], 'r--', lw=2)
    axes[1].set_xlabel('True CATE')
    axes[1].set_ylabel('Predicted CATE')
    axes[1].set_title('T-Learner: True vs Predicted')
    
    # 图3: CATE 分布对比
    axes[2].hist(true_cate, bins='auto', alpha=0.5, label='True CATE', density=True)
    axes[2].hist(s_pred, bins='auto', alpha=0.5, label='S-Learner', density=True)
    axes[2].hist(t_pred, bins='auto', alpha=0.5, label='T-Learner', density=True)
    axes[2].set_xlabel('CATE')
    axes[2].set_ylabel('Density')
    axes[2].set_title('CATE Distribution')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("请先完成上面的练习，生成预测结果")

---

### 参考答案: R-Learner 实现

<details>
<summary>点击查看参考答案</summary>

```python
class SimpleRLearner:
    def __init__(self):
        self.model_m = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model_e = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model_tau = RandomForestRegressor(n_estimators=100, random_state=42)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        # Step 1: 估计主效应
        self.model_m.fit(X, Y)
        self.model_e.fit(X, T)
        
        # Step 2: 计算残差
        Y_tilde = Y - self.model_m.predict(X)
        T_tilde = T - self.model_e.predict(X)
        
        # Step 3: 加权回归估计 CATE
        # 使用 |T_tilde| 作为权重，避免负权重
        weights = np.abs(T_tilde) + 1e-6  # 加小常数避免零权重
        self.model_tau.fit(X, Y_tilde / (T_tilde + 1e-6), sample_weight=weights)
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        return self.model_tau.predict(X)
```

</details>


---

### 参考答案: S-Learner 和 T-Learner 实现

<details>
<summary>点击查看参考答案</summary>

```python
# S-Learner 参考答案
class SimpleSLearner:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        # 合并 X 和 T
        X_with_T = np.column_stack([X, T.reshape(-1, 1)])
        self.model.fit(X_with_T, Y)
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        n = X.shape[0]
        # 构造 T=1 的特征
        X_with_T1 = np.column_stack([X, np.ones(n)])
        # 构造 T=0 的特征
        X_with_T0 = np.column_stack([X, np.zeros(n)])
        # 预测并计算差值
        Y1_pred = self.model.predict(X_with_T1)
        Y0_pred = self.model.predict(X_with_T0)
        return Y1_pred - Y0_pred

# T-Learner 参考答案
class SimpleTLearner:
    def __init__(self):
        self.model_0 = RandomForestRegressor(n_estimators=100, random_state=42)
        self.model_1 = RandomForestRegressor(n_estimators=100, random_state=42)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        # 按 T 分组
        mask_0 = (T == 0)
        mask_1 = (T == 1)
        # 分别训练
        self.model_0.fit(X[mask_0], Y[mask_0])
        self.model_1.fit(X[mask_1], Y[mask_1])
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        Y1_pred = self.model_1.predict(X)
        Y0_pred = self.model_0.predict(X)
        return Y1_pred - Y0_pred

# 数据生成参考答案
def generate_simple_uplift_data(
    n: int = 1000,
    heterogeneous: bool = True,
    seed: int = 42
) -> Tuple[pd.DataFrame, np.ndarray]:
    np.random.seed(seed)
    
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    T = np.random.binomial(1, 0.5, n)
    
    if heterogeneous:
        true_cate = 2 + X1 - 0.5 * X2
    else:
        true_cate = np.ones(n) * 2
    
    noise = np.random.randn(n) * 0.5
    Y0 = 5 + 2 * X1 + X2 + noise
    Y1 = Y0 + true_cate
    Y = T * Y1 + (1 - T) * Y0
    
    df = pd.DataFrame({
        'X1': X1,
        'X2': X2,
        'T': T,
        'Y': Y
    })
    
    return df, true_cate

# 评估函数参考答案
def evaluate_cate_estimation(
    true_cate: np.ndarray,
    predicted_cate: np.ndarray
) -> dict:
    mse = np.mean((true_cate - predicted_cate)**2)
    return {
        'MSE': mse,
        'MAE': np.mean(np.abs(true_cate - predicted_cate)),
        'Correlation': np.corrcoef(true_cate, predicted_cate)[0, 1],
        'PEHE': np.sqrt(mse)
    }
```

</details>


In [ ]:
# ============================================================
# 🎯 练习 1.6: 实现 R-Learner (进阶)
# ============================================================
# 目标: 实现 R-Learner，使用双重去偏方法直接优化 CATE
# 提示:
#   1. fit() 方法:
#      - 训练 model_m 预测 E[Y|X]
#      - 训练 model_e 预测 E[T|X]
#      - 计算残差 Y_tilde 和 T_tilde
#      - 使用加权回归训练 model_tau
#   2. predict_cate() 方法: 直接使用 model_tau 预测
# ============================================================

class SimpleRLearner:
    """
    R-Learner (Robinson Learner)
    
    核心思想:
    - Step 1: 估计 m(x) = E[Y|X] 和 e(x) = E[T|X]
    - Step 2: 计算残差 Y_tilde = Y - m(X), T_tilde = T - e(X)
    - Step 3: 最小化 (Y_tilde - T_tilde * tau(X))^2
    
    这是一种「双重去偏」的方法，直接优化 CATE 估计
    """
    
    def __init__(self):
        # 初始化三个模型
        self.model_m = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)  # E[Y|X]
        self.model_e = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=43)  # E[T|X]
        self.model_tau = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=44)  # tau(X)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 R-Learner
        
        TODO: 实现 R-Learner 的训练过程
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE
        
        TODO: 使用 model_tau 预测
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===

---

## Part 3: 进阶 Meta-Learners (可选)

R-Learner 和 DR-Learner 是更高级的方法，如果完成了基础练习可以尝试！

In [ ]:
# ============================================================
# 🎯 练习 1.7: 实现 DR-Learner (进阶)
# ============================================================
# 目标: 实现 DR-Learner，使用双重稳健方法估计 CATE
# 提示:
#   1. fit() 方法:
#      - 训练 model_0 和 model_1 分别预测两组的结果
#      - 训练 model_e 预测倾向得分
#      - 使用 AIPW 公式构造伪结果
#      - 训练 model_tau 预测伪结果
#   2. predict_cate() 方法: 使用 model_tau 预测
# ============================================================

class SimpleDRLearner:
    """
    DR-Learner (Doubly Robust Learner)
    
    核心思想:
    - Step 1: 分别估计 μ₀(x), μ₁(x), e(x)
    - Step 2: 构造伪结果 (Pseudo-Outcome) 使用 AIPW 公式
    - Step 3: 将伪结果回归到 X 上得到 CATE
    
    这种方法具有「双重稳健性」：
    - 如果结果模型 (μ₀, μ₁) 正确，估计一致
    - 如果倾向得分模型 (e) 正确，估计也一致
    """
    
    def __init__(self):
        # 初始化四个模型
        self.model_0 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)  # μ₀(X)
        self.model_1 = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=43)  # μ₁(X)
        self.model_e = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=44)  # e(X)
        self.model_tau = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=45)  # τ(X)
    
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """
        训练 DR-Learner
        
        TODO: 实现 DR-Learner 的训练过程
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===
        
        return self
    
    def predict_cate(self, X: np.ndarray) -> np.ndarray:
        """
        预测 CATE
        
        TODO: 使用 model_tau 预测
        """
        # === 你的代码开始 ===
        
        pass  # 删除此行，填入你的代码
        
        # === 你的代码结束 ===

---

## Part 9: Meta-Learner 选择指南

面对这么多 Meta-Learner，应该怎么选？这里提供一个实用的决策框架。

### 方法对比总结表

| Meta-Learner | 模型数量 | 核心思想 | 主要优势 | 主要劣势 | 适用场景 |
|--------------|---------|---------|---------|---------|----------|
| **S-Learner** | 1个 | 将 T 作为特征 | 简单，利用全部数据 | 可能低估效应（正则化偏差） | 效应小、数据少、两组相似 |
| **T-Learner** | 2个 | 分组建模 | 灵活，无正则化偏差 | 方差大（特别是不平衡时） | 效应大、两组差异大 |
| **X-Learner** | 4个 | 交叉估计 + 倾向得分加权 | 对不平衡数据友好 | 复杂，需要倾向得分 | 样本不平衡 |
| **R-Learner** | 3个 | 双重去偏，直接优化 CATE | 理论优雅，DML 框架 | 实现复杂 | 追求理论保证 |
| **DR-Learner** | 4个 | 双重稳健，伪结果 | 双保险，稳健性强 | 实现最复杂 | 模型不确定性大 |

### 决策树：应该用哪个 Meta-Learner？

```
开始
│
├─ 数据量很小 (n < 500) ?
│  └─ 是 → S-Learner (利用全部数据)
│  └─ 否 → 继续
│
├─ 处理组和控制组严重不平衡 (比如 90% vs 10%) ?
│  └─ 是 → X-Learner (倾向得分加权)
│  └─ 否 → 继续
│
├─ 担心模型设定错误？
│  └─ 是 → DR-Learner (双重稳健)
│  └─ 否 → 继续
│
├─ 需要理论保证和推断？
│  └─ 是 → R-Learner (DML 框架)
│  └─ 否 → T-Learner (简单有效)
```

### 实践建议

#### 1. 快速原型阶段

**推荐: T-Learner**
- 原因: 实现简单，性能稳定，易于理解
- 何时升级: 如果发现样本不平衡或效应很小

#### 2. 生产环境

**推荐策略: 组合使用**
- 同时训练 T-Learner 和 DR-Learner
- 用验证集选择更好的
- 或者集成两者的预测 (Ensemble)

#### 3. 研究/论文

**推荐: R-Learner 或 DR-Learner**
- 原因: 有理论保证，可以做统计推断
- 配合 Cross-Fitting 提高稳健性

### 关键因素分析

#### 因素 1: 样本量

| 样本量 | 推荐方法 | 理由 |
|--------|---------|------|
| n < 500 | S-Learner | 需要利用全部数据 |
| 500 < n < 5000 | T-Learner | 平衡 bias-variance |
| n > 5000 | R/DR-Learner | 可以承受更复杂的模型 |

#### 因素 2: 样本平衡性

| 处理组占比 | 推荐方法 | 理由 |
|-----------|---------|------|
| 20% - 80% | T-Learner | 两组样本都足够 |
| 10% - 20% 或 80% - 90% | X-Learner | 需要倾向得分加权 |
| < 10% 或 > 90% | DR-Learner | 双重稳健性重要 |

#### 因素 3: 效应大小

| 效应大小 | 推荐方法 | 理由 |
|---------|---------|------|
| 很小 (< 5% 提升) | S-Learner 或 DR-Learner | 需要低方差 |
| 中等 (5-20%) | T-Learner | 标准选择 |
| 很大 (> 20%) | T-Learner | 两组差异明显 |

#### 因素 4: 异质性程度

| 异质性 | 推荐方法 | 理由 |
|--------|---------|------|
| 低 (CATE 接近常数) | S-Learner | 不需要复杂建模 |
| 中 | T-Learner | 可以捕捉部分异质性 |
| 高 (CATE 变化剧烈) | R/DR-Learner + 非线性模型 | 需要灵活性 |

### 组合策略 (Ensemble)

不确定用哪个？可以组合多个！

```python
# 简单平均
tau_ensemble = (tau_s + tau_t + tau_dr) / 3

# 加权平均 (用验证集确定权重)
tau_ensemble = w1 * tau_t + w2 * tau_dr

# Stacking (用另一个模型学习如何组合)
meta_model = RandomForestRegressor()
meta_model.fit(
    np.column_stack([tau_s, tau_t, tau_r, tau_dr]),
    true_cate  # 仅在有真实标签时
)
```

### 常见误区

| 误区 | 正确理解 |
|------|----------|
| "DR-Learner 最好，应该总是用它" | DR-Learner 复杂度高，小数据可能过拟合 |
| "S-Learner 太简单，不应该用" | 小数据或低异质性时，S-Learner 可能更好 |
| "选一个方法一劳永逸" | 应该多试几个，用验证集选择 |
| "PEHE 低就说明模型好" | PEHE 需要真实 CATE，实际应用中没有 |

### 实际应用中的验证方法

因为实际应用中我们没有真实的 CATE，怎么验证？

#### 方法 1: 分组 ATE 一致性检查

```python
# 按预测的 CATE 分组
quartiles = pd.qcut(predicted_cate, q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

# 每组计算实际 ATE
for q in ['Q1', 'Q2', 'Q3', 'Q4']:
    mask = (quartiles == q)
    ate = df[mask & (df['T']==1)]['Y'].mean() - df[mask & (df['T']==0)]['Y'].mean()
    print(f"{q}: Predicted CATE = {predicted_cate[mask].mean():.2f}, Actual ATE = {ate:.2f}")
    
# 期望: Q4 的 ATE > Q3 > Q2 > Q1 (单调性)
```

#### 方法 2: Qini 曲线 / Uplift 曲线

见下一个练习: Uplift Evaluation

#### 方法 3: 业务 A/B 测试验证

- 在预测 CATE 高的人群上做 A/B 测试
- 验证效应是否确实更大


## 🎯 Meta-Learner 面试高频题

### Q1: S-Learner 和 T-Learner 的本质区别是什么？

**标准答案**:

**S-Learner**:
- 单一模型: $\hat{\mu}(x, t) = f(x, t)$
- 将 T 当作普通特征
- 问题: **正则化偏差** - 处理效应容易被压缩

**T-Learner**:
- 两个独立模型: $\hat{\mu}_0(x), \hat{\mu}_1(x)$
- 分别建模两组
- 问题: **高方差** - $Var[\hat{\tau}] = Var[\hat{\mu}_1] + Var[\hat{\mu}_0]$

**何时选择**:
- S-Learner: 数据少、效应小、两组相似
- T-Learner: 数据多、效应大、两组差异显著

### Q2: X-Learner 解决了什么问题？

**标准答案**:

X-Learner 主要解决 **样本不平衡** 问题 (如 90% vs 10%)。

**核心创新**:
1. 计算伪效应 (imputed treatment effects)
2. 用倾向得分加权组合两个估计

**数学**:
- 处理组伪效应: $D^1_i = Y_i - \hat{\mu}_0(X_i)$
- 控制组伪效应: $D^0_j = \hat{\mu}_1(X_j) - Y_j$
- 最终估计: $\hat{\tau}(x) = e(x) \cdot \hat{\tau}_0(x) + (1-e(x)) \cdot \hat{\tau}_1(x)$

**直觉**: 在样本多的组上，估计更可靠，给予更高权重。

### Q3: R-Learner 的核心思想是什么？

**标准答案**:

R-Learner 的核心是 **双重去偏** (double residualization)。

**Robinson 分解**:
$$Y = m(X) + (T - e(X)) \cdot \tau(X) + \epsilon$$

**两步法**:
1. 估计并去除主效应: $\tilde{Y} = Y - \hat{m}(X)$, $\tilde{T} = T - \hat{e}(X)$
2. 直接优化 CATE: $\min_{\tau} \sum (\tilde{Y}_i - \tilde{T}_i \cdot \tau(X_i))^2$

**优势**:
- 避免正则化偏差
- 直接优化 CATE 的 MSE
- 有 DML 理论保证 ($\sqrt{n}$-一致性)

### Q4: 什么是 Honest Splitting？为什么因果森林需要它？

**标准答案**:

**Honest Splitting** 是将样本分为两部分:
- **分裂样本** (50%): 用于构建树结构
- **估计样本** (50%): 用于叶节点的 CATE 估计

**为什么需要**:
1. **避免过拟合**: 如果用同一批数据既构建树又估计，会过度适应数据
2. **统计性质**: 保证渐近正态性，使置信区间有效
3. **无偏估计**: 分裂和估计独立，消除选择偏差

**类比**: 就像回测投资策略时，不能用训练数据验证效果。

### Q5: 如何评估 Uplift 模型？Qini 曲线怎么解读？

**标准答案**:

**Qini 曲线**:
$$Qini(k) = Y_t(k) - Y_c(k) \times \frac{n_t(k)}{n_c(k)}$$

**解读**:
- X 轴: 干预比例 (按预测 CATE 从高到低)
- Y 轴: 累积增量收益
- 曲线越高越好

**AUUC (Area Under Uplift Curve)**:
- Qini 曲线下的面积
- 类似 AUC，但针对 Uplift

**其他评估方法**:
1. **Uplift by Decile**: 分 10 组，验证单调性
2. **Top k% Uplift**: 高分组的实际 uplift
3. **A/B 测试验证**: 在预测高 CATE 人群上实验

### Q6: DR-Learner 的「双重稳健性」是什么意思？

**标准答案**:

**双重稳健性** (Double Robustness): 只要以下之一正确，估计就是一致的:
1. 结果模型正确: $\hat{\mu}_0, \hat{\mu}_1$
2. 倾向得分模型正确: $\hat{e}(x)$

**AIPW 公式**:
$$\tilde{\tau}_i = \frac{T_i(Y_i - \hat{\mu}_1(X_i))}{\hat{e}(X_i)} - \frac{(1-T_i)(Y_i - \hat{\mu}_0(X_i))}{1-\hat{e}(X_i)} + \hat{\mu}_1(X_i) - \hat{\mu}_0(X_i)$$

**直觉**:
- 结果模型提供基础估计
- IPW 修正提供偏差校正
- 两者互补，提供"双保险"

**实践价值**: 当不确定哪个模型更好时，DR 是稳健选择。

---

## 更新后的思考题

请在下面的 markdown 单元格中写下你的答案：

### 1. R-Learner 的「双重去偏」是什么意思？为什么需要去偏？

**你的答案:**


### 2. DR-Learner 的「双重稳健性」是什么意思？为什么说它是「双保险」？

**你的答案:**


### 3. 如果你有 10000 个样本，处理组和控制组比例是 50:50，效应大小未知，你会选择哪个 Meta-Learner？为什么？

**你的答案:**


### 4. R-Learner 和 T-Learner 的主要区别是什么？

**你的答案:**


### 5. 在实际应用中（没有真实 CATE），你如何验证 Meta-Learner 的预测质量？

**你的答案:**


### 6. (进阶) DR-Learner 中的伪结果 (Pseudo-Outcome) 为什么是无偏的？

**你的答案:**

